本节的验证码识别是基于tensorflow官方的models中slim中的alexnet，基于slim/nets/alexnet.py文件中的网络结构，来实现验证码识别的多任务学习。

具体的源文件，可以在Lec 09中的HandWriteImageClassification的slim文件夹下面查看。

alexnet.py中主要定义了两个函数：
- alexnet_v2_arg_scope
- alexnet_v2

alexnet_v2_arg_scope定义的是里面网络使用的参数，具体内容需要查看源码或官方文档。

### alexnet_v2是定义了alex_net的网络结构

**参数：**
- inputs: 
    - a tensor of size [batch_size, height, width, channels].
- num_classes: 
    - the number of predicted classes. 预测的目标的类别数，如源码中是识别1000种不同的图片类型（如inception_v3）。
    - 在验证码识别中需要将其改成10（验证码由0-9十个数字组成，每个task的目标分类为10种）
- is_training: 
    - whether or not the model is being trained.
- dropout_keep_prob: 
    - the probability that activations are kept in the dropout layers during training.
- spatial_squeeze: 
    - whether or not should squeeze the spatial dimensions of thelogits. Useful to remove unnecessary dimensions for classification. 指定或将全部的维度中，维度为1的维度去除。
- scope: 
    - Optional scope for the variables.
- global_pool: 
    - Optional boolean flag. If True, the input to the classification layer is avgpooled to size 1x1, for any input size. (This is not part of the original AlexNet.)



**注意:**

All the fully_connected layers have been transformed to conv2d layers.
所有的全连接层都已经被转化成了conv2d（卷积层）。

To use in classification mode, resize input to 224x224 or set global_pool=True. 
使用分类器，将input的shape设置为224x224，或者设置global_pool=True。

To use in fully convolutional mode, set spatial_squeeze to false.
在卷积模式下，设置spatial_squeeze=False, 不要将任何维度的1维去除。

The LRN layers have been removed and change the initializers from random_normal_initializer to xavier_initializer.
LRN层已经被移除，初始化从random_normal_intializer转换成了xavier_innitalizer.

**返回值:**
- net: 
    - the output of the logits layer (if num_classes is a non-zero integer), or the non-dropped-out input to the logits layer (if num_classes is 0 or None).
- end_points: 
    - a dict of tensors with intermediate activations.


查看下面的alexnet_v2函数的定义方式，可以发现：
- inputs是[batch_size, height, width, channels]的张量；
- num_classes的默认值是1000，在验证码识别调用的时候，要将num_classes传参为10（如果验证码的字符串构成的内容更多，做相应的更改）；   
- is_training 默认为True
- dropout_keep_prob 默认为0.5
- spatial_squeeze 默认为True，会去除掉该张量中为1的维度。
- scope 范围，是nets_factory对各个不同的网络的管理，相应的参数通过这种方式确定
- global_pool 默认值为False


def alexnet_v2(inputs,
               num_classes=1000,
               is_training=True,
               dropout_keep_prob=0.5,
               spatial_squeeze=True,
               scope='alexnet_v2',
               global_pool=False):
               
  with tf.variable_scope(scope, 'alexnet_v2', [inputs]) as sc:
    end_points_collection = sc.original_name_scope + '_end_points'
    # Collect outputs for conv2d, fully_connected and max_pool2d.
    with slim.arg_scope([slim.conv2d, slim.fully_connected, slim.max_pool2d],
                        outputs_collections=[end_points_collection]):
      net = slim.conv2d(inputs, 64, [11, 11], 4, padding='VALID',
                        scope='conv1')
      net = slim.max_pool2d(net, [3, 3], 2, scope='pool1')
      net = slim.conv2d(net, 192, [5, 5], scope='conv2')
      net = slim.max_pool2d(net, [3, 3], 2, scope='pool2')
      net = slim.conv2d(net, 384, [3, 3], scope='conv3')
      net = slim.conv2d(net, 384, [3, 3], scope='conv4')
      net = slim.conv2d(net, 256, [3, 3], scope='conv5')
      net = slim.max_pool2d(net, [3, 3], 2, scope='pool5')

      # Use conv2d instead of fully_connected layers.
      with slim.arg_scope([slim.conv2d],
                          weights_initializer=trunc_normal(0.005),
                          biases_initializer=tf.constant_initializer(0.1)):
        net = slim.conv2d(net, 4096, [5, 5], padding='VALID',
                          scope='fc6')
        net = slim.dropout(net, dropout_keep_prob, is_training=is_training,
                           scope='dropout6')
        net = slim.conv2d(net, 4096, [1, 1], scope='fc7')
        # Convert end_points_collection into a end_point dict.
        end_points = slim.utils.convert_collection_to_dict(
            end_points_collection)
        if global_pool:
          net = tf.reduce_mean(net, [1, 2], keep_dims=True, name='global_pool')
          end_points['global_pool'] = net
        if num_classes:
          net = slim.dropout(net, dropout_keep_prob, is_training=is_training,
                             scope='dropout7')
          net = slim.conv2d(net, num_classes, [1, 1],
                            activation_fn=None,
                            normalizer_fn=None,
                            biases_initializer=tf.zeros_initializer(),
                            scope='fc8')
          if spatial_squeeze:
            net = tf.squeeze(net, [1, 2], name='fc8/squeezed')
          end_points[sc.name + '/fc8'] = net
      return net, end_points
alexnet_v2.default_image_size = 224



## 更改alexnet.py
主要是将验证码识别的多任务设计到alexnet网络中

具体的比较或在tensorboard中查看graph，需要另外设置。